# Analyzing Chess Games Using the Chess.com API in Python

## The purpose of this script is to export monthly games to a CSV format

In [ ]:
# Chess.com API library
from chessdotcom import get_player_game_archives, get_player_games_by_month, Client

# Path Library
from pathlib import Path

# Displays the JSON in a nicer more readable format
from pprint import pprint

# DataFrame Library
import pandas as pd

# Datetime Library
import datetime as dt

In [ ]:
# This is what provides access to the chess.com API. You need this to be able to access the API.
Client.request_config["headers"]["User-Agent"] = (
   "My Game Analysis. "

   # Replace with your email (not necessary but important if you happen to hit your scrape rate limit)
   "Contact me at example@gmail.com"
)

In [ ]:
# Replace with your username or another public chess.com username to analyze someone else's games
game_archives = get_player_game_archives("my_username")

# Looking at the archives let's us know what year's and month's we have played
game_archives.json["archives"]

### The following two cells below are a demo of how to extract JSON metadata from a particular month and convert it into a DataFrame.

In [ ]:
# Gets player games by month
december_games = get_player_games_by_month("my_username", year = 2023, month = 12)

# Raw JSON data for the month of December 2023 for the first entry in the list
pprint(december_games.json["games"][0])

In [ ]:
december_df = pd.json_normalize(december_games.json["games"])

# First 5 rows of the dataframe
december_df.head()

In [ ]:
def get_monthly_games(your_username, game_year, game_month):
    """This function will export your monthly games to a dataframe from JSON format.

    Args:
        your_username (str): your account username on chess.com or someone else's if you want to analyze someone else's games
        game_year (int): the year
        game_month (int): the month

    Returns:
        pandas.DataFrame : Returns a dataframe of the normalized JSON data.
    """
    games = get_player_games_by_month(username = your_username, year = game_year, month = game_month)
    game_df = pd.json_normalize(games.json["games"])
    return game_df

In [ ]:
def save_games(username, game_year_interval):
    """ Saves games based on game year interval

    Args:
        username (str): your username
        game_year_interval (tuple/list): game years to get games from
    """
    data_dir = Path.cwd() / "data" # the path where the data will be stored
    if not data_dir.exists():
        data_dir.mkdir(parents = True, exist_ok = True)
    games_list = []
    for game_year in range(*game_year_interval):
        for game_month in range(1, 13):
            if dt.date(year = game_year, month = game_month, day = 1) < dt.date.today():
                games_list.append(get_monthly_games(username,game_year,game_month))
    
    games_df = pd.concat(games_list)
    games_df.to_csv(data_dir / "chess_games.csv", index = False)

In [ ]:
save_games("your_username", [2023, dt.date.today().year])